In [61]:
import pandas as pd

In [62]:
csv_1 = pd.read_csv('Contrastive.csv', index_col=0, sep='|')

In [63]:
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The most likely answer is ""contradiction"". ..."
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The most likely answer is ""neutral"". The pr..."
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The most likely answer is ""entailment"". The..."
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The most likely answer is ""entailment"". The..."
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The most likely answer is ""entailment"". The..."
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,"8. The most likely answer is ""neutral"". The pr..."
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,"9. The most likely answer is ""entailment"". The..."
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,"10. The most likely answer is ""entailment"". Th..."
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,Premise: Three naked little boys are playing i...


In [470]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [64]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
For the premise '%s' and hypothesis %s, among the choices "entailment", "neutral", "contradiction", which is the most likely answer? Highlight what sets your choice apart from the others.\n
'''

In [65]:
error = csv_1[csv_1.rationales.isna()]

In [66]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

In [67]:
error

,premise,hypothesis,rationales,prompt


In [68]:
print(error.prompt[0:].values)

[]


In [69]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [70]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [71]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [74]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [76]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [77]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,split,correct_index,LLM_answer


In [481]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [78]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [79]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [80]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [81]:
paper.shape

(5592, 6)

In [82]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The most likely answer is ""contradiction"". ...","is ""contradiction"". the premise describes a m...",{'contradiction': 5},contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The most likely answer is ""neutral"". The pr...","is ""neutral"". the premise describes two girls...",{'neutral': 5},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The most likely answer is ""entailment"". The...","is ""entailment"". the premise describes a dog ...",{'entailment': 5},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The most likely answer is ""entailment"". The...","is ""entailment"". the premise describes a bald...",{'entailment': 5},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The most likely answer is ""entailment"". The...","is ""entailment"". the premise describes a boy ...",{'entailment': 5},entailment
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,"8. The most likely answer is ""neutral"". The pr...","is ""neutral"". the premise describes three wom...",{'neutral': 5},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,"9. The most likely answer is ""entailment"". The...","is ""entailment"". the premise states that a ba...",{'entailment': 5},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,"10. The most likely answer is ""entailment"". Th...","is ""entailment"". the premise describes two ki...","{'entailment': 5, 'contradicts': 89}",entailment


In [83]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'

In [84]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The most likely answer is ""contradiction"". ...","is ""contradiction"". the premise describes a m...",{'contradiction': 5},contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The most likely answer is ""neutral"". The pr...","is ""neutral"". the premise describes two girls...",{'neutral': 5},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The most likely answer is ""entailment"". The...","is ""entailment"". the premise describes a dog ...",{'entailment': 5},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The most likely answer is ""entailment"". The...","is ""entailment"". the premise describes a bald...",{'entailment': 5},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The most likely answer is ""entailment"". The...","is ""entailment"". the premise describes a boy ...",{'entailment': 5},entailment
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,"8. The most likely answer is ""neutral"". The pr...","is ""neutral"". the premise describes three wom...",{'neutral': 5},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,"9. The most likely answer is ""entailment"". The...","is ""entailment"". the premise states that a ba...",{'entailment': 5},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,"10. The most likely answer is ""entailment"". Th...","is ""entailment"". the premise describes two ki...","{'entailment': 5, 'contradicts': 89}",entailment


In [85]:
paper['my_label'] = total_df['LLM_answer']

In [86]:
sum(paper.label == paper.my_label) / len(paper)

0.6092632331902719

In [87]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [566]:
total_df.to_csv('contrastive - full.csv')

In [ ]:
csv_1.to_csv('if_else - full.csv', index=False)

In [568]:
pd.read_csv('/home/huy/Downloads/train_rationales.csv', delimiter='|')

,Unnamed: 0,premise,hypothesis
0,0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.
1,1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.
2,2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.
3,3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet
4,4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.
...,...,...,...
5587,5587,Three women walk down a city street; one looks...,Three women chat at home.
5588,5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...
5589,5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.
5590,5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.


: 